# Water Maze
https://github.com/vjann/DLC

### Description:
INSERT DESCRIPTION HERE

## Create a new project

Keep the projects separate. This function creates a new project with subdirectories and a basic configuration file under DeepLabCut directory.
You can add new videos to the project at any stage of the project. 

In [1]:
import datetime
import deeplabcut
from time import sleep

In [2]:
task='watermaze' # Enter the name of your experiment Task
experimenter='vj' # Enter the name of the experimenter
video=[r"C:\Users\vjj14\Downloads\watermazevideos\50909.mp4", r"C:\Users\vjj14\Downloads\watermazevideos\50911.mp4"] # Enter the paths of your videos you want to grab frames from.
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory= r'C:\Users\vjj14\Desktop\DeepLabCut') #change the working directory to where you want the folders created.
# The function returns the path, where your project is. 

Created "C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\videos"
Created "C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\labeled-data"
Created "C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\training-datasets"
Created "C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\dlc-models"
Creating the symbolic link of the video
Created the symlink of C:\Users\vjj14\Downloads\watermazevideos\50909.mp4 to C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\videos\50909.mp4
Created the symlink of C:\Users\vjj14\Downloads\watermazevideos\50911.mp4 to C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\videos\50911.mp4
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\videos\50909.mp4
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\videos\50911.mp4
Generated "C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-12\config.yaml"

A new project with name watermaze-vj-2019-07-12 is created at C:\Users\vjj14\Desktop\DeepLabCut and a 

In [5]:
path_config_file=r"C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\config.yaml"
project_name='watermaze'
def write_log(text, new_line=True):
    time_log= open("{0}log{1}.txt".format(path_config_file[:-11], project_name),"a+")
    if new_line:
        time_log.write("\n")
    time_log.write(text)
    time_log.close()
    print("log written")
    
def prompt_with_timeout(prompt_message):
  print('Waiting... please press Ctrl-C when you wish to proceed.')
  try:
    for i in range(0, 2*60): # 2 minutes
      sleep(1)
    return "No Input Received"
  except KeyboardInterrupt:
    return input(prompt_message)

## Extract frames from videos 
Select diverse frames, which are typical for the behavior you study that should be labeled.
(algo=='uniform') selects N frames either uniformly sampled from a particular video (or folder) . Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.
Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later, crop the frames to remove unnecessary parts.
Always check the output of cropping.

In [9]:
deeplabcut.add_new_videos(path_config_file, [r"C:\Users\vjj14\Downloads\watermazevideos\66019v2.mp4"])

Creating the symbolic link of the video
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.


In [8]:
?deeplabcut.add_new_videos

In [10]:
start = datetime.datetime.now()
write_log(str(start) + "***extract_frames start")

%matplotlib inline
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

end = datetime.datetime.now()
write_log(str(end) + "***extract_frames end")
write_log("   time elapsed:" + str(end-start))
write_log("     " + prompt_with_timeout("number of frames selected?") + "frames labeled")

log written
Config file read successfully.
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Downloads\watermazevideos\50909.mp4 ?
yes/no
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Downloads\watermazevideos\50911.mp4 ?
yes/no
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Downloads\watermazevideos\66019v2.mp4 ?
yes/noyes
Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 65.33  seconds.

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).
log written
log written
Waiting... please press Ctrl-C when you wish to proceed.
number of frames selected?i think 25
log written


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [11]:
start = datetime.datetime.now()
write_log(str(start) + "***label_frames start")

%gui wx
deeplabcut.label_frames(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***label_frames end")
write_log("   time elapsed:" + str(end-start))

log written
You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.
log written
log written


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. 

In [ ]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [12]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

log written
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\training-datasets\iteration-0\UnaugmentedDataSet_watermazeJul2  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1//train  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
log written
log written


In [ ]:
?deeplabcut.create_training_dataset

## Start training - If you want to use a CPU, continue. 
### If you want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [13]:
start = datetime.datetime.now()
write_log(str(start) + "***train_network start")
write_log("   DESCRIPTION:" + prompt_with_timeout("Training description(Network, Iterations, etc) "))

deeplabcut.train_network(path_config_file, saveiters=100, displayiters=50, maxiters=10001)

end = datetime.datetime.now()
write_log(str(end) + "***train_network end")
write_log("   time elapsed:" + str(end-start))

log written
Waiting... please press Ctrl-C when you wish to proceed.
Training description(Network, Iterations, etc) try again with edited video
log written


Config:
{'all_joints': [[0]],
 'all_joints_names': ['mouse'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\watermaze_vj90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\Documentation_data-watermaze_90shuffle1.pickle',
 'min_input_size': 64,
 'minsize': 100,
 'mirror': Fa

INFO:tensorflow:Restoring parameters from C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 10001
Display_iters overwritten as 50
Save_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\watermaze-vj-2019-07-02\\dlc-models\\iteration-0\\watermazeJul2-trainset90shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'd

iteration: 50 loss: 0.0669 lr: 0.005
iteration: 100 loss: 0.0221 lr: 0.005
iteration: 150 loss: 0.0159 lr: 0.005
iteration: 200 loss: 0.0092 lr: 0.005
iteration: 250 loss: 0.0100 lr: 0.005
iteration: 300 loss: 0.0069 lr: 0.005
iteration: 350 loss: 0.0093 lr: 0.005
iteration: 400 loss: 0.0076 lr: 0.005
iteration: 450 loss: 0.0072 lr: 0.005
iteration: 500 loss: 0.0075 lr: 0.005
iteration: 550 loss: 0.0068 lr: 0.005
iteration: 600 loss: 0.0059 lr: 0.005
iteration: 650 loss: 0.0071 lr: 0.005
iteration: 700 loss: 0.0055 lr: 0.005
iteration: 750 loss: 0.0069 lr: 0.005
iteration: 800 loss: 0.0063 lr: 0.005
iteration: 850 loss: 0.0053 lr: 0.005
iteration: 900 loss: 0.0051 lr: 0.005
iteration: 950 loss: 0.0057 lr: 0.005
iteration: 1000 loss: 0.0051 lr: 0.005
iteration: 1050 loss: 0.0052 lr: 0.005
iteration: 1100 loss: 0.0062 lr: 0.005
iteration: 1150 loss: 0.0051 lr: 0.005
iteration: 1200 loss: 0.0054 lr: 0.005
iteration: 1250 loss: 0.0053 lr: 0.005
iteration: 1300 loss: 0.0049 lr: 0.005
iterat

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.
log written
log written


In [ ]:
?deeplabcut.train_network

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
start = datetime.datetime.now()
write_log(str(start) + "***evaluate_network start")

deeplabcut.evaluate_network(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***evaluate_network end")
write_log("   time elapsed:" + str(end-start))

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [15]:
# videofile_path = [r"C:\Users\vjj14\Downloads\compressed25.mp4"] #Enter the list of videos to analyze.
videos=[[r"C:\Users\vjj14\Downloads\watermazevideos\50909.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\50911.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\50944v2.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\56199.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\56242v2.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\56244.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\63410v2.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\63447.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\66019v2.mp4"], [r"C:\Users\vjj14\Downloads\watermazevideos\66020v2.mp4"]]
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***analyze_videos start")
    write_log("   VIDEO: " + str(videofile_path))

    deeplabcut.analyze_videos(path_config_file,videofile_path, save_as_csv=True)

    end = datetime.datetime.now()
    write_log(str(end) + "***analyze_videos end")
    write_log("   time elapsed:" + str(end-start))

log written
log written


Config:
{'all_joints': [[0]],
 'all_joints_names': ['mouse'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\watermaze_vj90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\Documentation_data-watermaze_90shuffle1.pickle',
 'min_input_size': 64,
 'minsize': 100,
 'mirror': Fa

Using snapshot-10001 for model C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1\train\snapshot-10001
Starting to analyze %  C:\Users\vjj14\Downloads\watermazevideos\50909.mp4
Loading  C:\Users\vjj14\Downloads\watermazevideos\50909.mp4
Duration of video [s]:  65.43 , recorded with  28.92 fps!
Overall # of frames:  1892  found with (before cropping) frame dimensions:  800 600
Starting to extract posture



 91%|███████████████████████████████████████▎   | 1728/1892 [00:40<00:03, 43.09it/s]


100%|██████████████████████████████████████████▉| 1890/1892 [00:43<00:00, 43.30it/s]
1908it [00:44, 44.77it/s]                                                           

...

Config:
{'all_joints': [[0]],
 'all_joints_names': ['mouse'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\watermaze_vj90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_watermazeJul2\\Documentation_data-watermaze_90shuffle1.pickle',
 'min_input_size': 64,
 'minsize': 100,
 'mirror': Fa

Using snapshot-10001 for model C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\dlc-models\iteration-0\watermazeJul2-trainset90shuffle1\train\snapshot-10001
Starting to analyze %  C:\Users\vjj14\Downloads\watermazevideos\66020v2.mp4
Loading  C:\Users\vjj14\Downloads\watermazevideos\66020v2.mp4
Duration of video [s]:  64.67 , recorded with  30.0 fps!
Overall # of frames:  1940  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture



 94%|████████████████████████████████████████▍  | 1824/1940 [02:22<00:09, 12.66it/s]


100%|██████████████████████████████████████████▉| 1938/1940 [02:31<00:00, 12.99it/s]
1957it [02:32, 12.66it/s]                                                           

Detected frames:  1940
Saving results in C:\Users\vjj14\Downloads\watermazevideos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!
log written
log written


In [ ]:
?deeplabcut.analyze_videos

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
alg = 'uncertain'
param = 0.0002 #epsilon or p_bound

start = datetime.datetime.now()
write_log(str(start) + "***extract_outlier_frames start")

deeplabcut.extract_outlier_frames(path_config_file, videofile_path, outlieralgorithm=alg, p_bound=param)
num_frames = prompt_with_timeout("number of frames selected?")
write_log("   VIDEOS: {0}, ALGORITHM: {1}, PARAMETER: {2}, NUM FRAMES: {3}".format(str(videofile_path), alg, param, num_frames))

end = datetime.datetime.now()
write_log(str(end) + "***extract_outlier_frames start")
write_log("   time elapsed:" + str(end-start))

In [ ]:
?deeplabcut.extract_outlier_frames

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

%gui wx
deeplabcut.refine_labels(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))


In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
start = datetime.datetime.now()
write_log(str(start) + "***merge_datasets start")

deeplabcut.merge_datasets(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***merge_datasets end")
write_log("   time elapsed:" + str(end-start))

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [16]:
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***create_labeled_video start")
    write_log("   VIDEOS: {}".format(str(videofile_path)))

    deeplabcut.create_labeled_video(path_config_file,videofile_path)

    end = datetime.datetime.now()
    write_log(str(end) + "***create_labeled_video end")
    write_log("   time elapsed:" + str(end-start))

log written
log written
Starting %  C:\Users\vjj14\Downloads\watermazevideos ['C:\\Users\\vjj14\\Downloads\\watermazevideos\\50909.mp4']
Loading  C:\Users\vjj14\Downloads\watermazevideos\50909.mp4 and data.
False 0 800 0 600
1892
Duration of video [s]:  65.43 , recorded with  28.92 fps!
Overall # of frames:  1892 with cropped frame dimensions:  800 600
Generating frames and creating video.



100%|██████████████████████████████████████████| 1892/1892 [00:08<00:00, 229.90it/s]

log written
log written
log written
log written
Starting %  C:\Users\vjj14\Downloads\watermazevideos ['C:\\Users\\vjj14\\Downloads\\watermazevideos\\50911.mp4']
Loading  C:\Users\vjj14\Downloads\watermazevideos\50911.mp4 and data.
False 0 800 0 600
1927
Duration of video [s]:  65.46 , recorded with  29.44 fps!
Overall # of frames:  1927 with cropped frame dimensions:  800 600
Generating frames and creating video.



100%|██████████████████████████████████████████| 1927/1927 [00:08<00:00, 231.11it/s]

log written
log written
log written
log written
Starting %  C:\Users\vjj14\Downloads\watermazevideos ['C:\\Users\\vjj14\\Downloads\\watermazevideos\\50944v2.mp4']
Loading  C:\Users\vjj14\Downloads\watermazevideos\50944v2.mp4 and data.
False 0 1920 0 1080
1966
Duration of video [s]:  65.53 , recorded with  30.0 fps!
Overall # of frames:  1966 with cropped frame dimensions:  1920 1080
Generating frames and creating video.


...

log written
log written
log written
log written
Starting %  C:\Users\vjj14\Downloads\watermazevideos ['C:\\Users\\vjj14\\Downloads\\watermazevideos\\66020v2.mp4']
Loading  C:\Users\vjj14\Downloads\watermazevideos\66020v2.mp4 and data.
False 0 1920 0 1080
1940
Duration of video [s]:  64.67 , recorded with  30.0 fps!
Overall # of frames:  1940 with cropped frame dimensions:  1920 1080
Generating frames and creating video.



 36%|███████████████▊                            | 696/1940 [00:10<00:18, 66.62it/s]


 72%|███████████████████████████████            | 1401/1940 [00:21<00:07, 68.31it/s]


100%|███████████████████████████████████████████| 1940/1940 [00:29<00:00, 64.76it/s]

log written
log written


In [ ]:
?deeplabcut.create_labeled_video

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
videofile_path

In [ ]:
%matplotlib notebook 
#for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
?deeplabcut.plot_trajectories

In [18]:
import numpy as np
import pandas as pd

## Pandas analysis of Data

In [19]:
dfs = []
times = []
mouse_number = []
frames = []
frame_rate = []

m50909 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\50909DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m50909)
times.append(65.4)
mouse_number.append(50909)
frame_rate.append(28.92)

m50911 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\50911DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m50911)
times.append(65.45)
mouse_number.append(50911)
frame_rate.append(29.97)

m50944 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\50944v2DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m50944)
times.append(65.58)
mouse_number.append(50944)
frame_rate.append(30)

m56199 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\56199DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m56199)
times.append(65.46)
mouse_number.append(56199)
frame_rate.append(28.9)

m56242 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\56242v2DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m56242)
times.append(64.26)
mouse_number.append(56242)
frame_rate.append(30)

m56244 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\56244DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m56244)
times.append(65.44)
mouse_number.append(56244)
frame_rate.append(29.36)

m63410 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\63410v2DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m63410)
times.append(65.09)
mouse_number.append(63410)
frame_rate.append(30)

m63447 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\63447DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m63447)
times.append(67.49)
mouse_number.append(63447)
frame_rate.append(27.18)

m66019 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\66019v2DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m66019)
times.append(65.39)
mouse_number.append(66019)
frame_rate.append(30)

m66020 = pd.read_hdf(r"C:\Users\vjj14\Downloads\watermazevideos\66020v2DeepCut_resnet50_watermazeJul2shuffle1_10001.h5")
dfs.append(m66020)
times.append(64.73)
mouse_number.append(66020)
frame_rate.append(30)

for i in range(len(dfs)):
    dfs[i].columns = dfs[i].columns.droplevel()
    dfs[i] = dfs[i]['mouse']
frames = [x.shape[0] for x in dfs]
dfs[-1].describe()

coords,x,y,likelihood
count,1940.000000,1940.000000,1940.000000
mean,946.295723,697.295266,0.999270
std,127.895095,168.633964,0.000726
min,641.997136,264.024598,0.990851
25%,869.341200,606.285450,0.999114
50%,941.118711,704.658859,0.999493
75%,1015.347725,827.770800,0.999704
max,1269.114223,967.732965,0.999957


In [20]:
dfs[0].describe()

coords,x,y,likelihood
count,1892.000000,1892.000000,1892.000000
mean,448.484442,472.630454,0.999518
std,78.063852,58.596374,0.000454
min,329.984744,189.983248,0.994951
25%,377.552786,453.853230,0.999378
50%,442.812777,488.722864,0.999665
75%,524.898930,507.508077,0.999798
max,575.531316,532.620696,0.999959


In [45]:
test = dfs[4]
print(test.head())
def inRightQuadrant(row):
    x = row['x']
    y = row['y']
    if 182*x + 219*y - 142686 > 0 and 432*x - 347*y - 77046 < 0:
        return 1
    return 0
def inRightQuadrant2(row):
    x = row['x']
    y = row['y']
    if 777*x - 625*y - 436029 < 0 and 646*x + 781*y - 1069355 > 0:
        return 1
    return 0
v2_indexes = [2, 4, 6, 8, 9]
proportion_spent = []
for i in range(len(dfs)):   
#     dfs[i].plot(kind='scatter', x = 'x', y= 'y')
    if i in v2_indexes:
        dfs[i]['rightQuadrant'] = dfs[i].apply(inRightQuadrant2, axis=1)
    else:
        dfs[i]['rightQuadrant'] = dfs[i].apply(inRightQuadrant, axis=1)
    
#     dfs[i][dfs[i]['rightQuadrant']].plot(kind='scatter', x = 'x', y= 'y')
#     df.reset_index().plot(kind='scatter', x='index', y='rightQuadrant')
    proportion_spent.append(np.sum(dfs[i]['rightQuadrant']) / dfs[i].shape[0])
print(proportion_spent)


coords           x           y  likelihood  rightQuadrant
0       784.988801  314.837857    0.999547              0
1       784.988798  314.714167    0.999550              0
2       784.988808  314.714165    0.999550              0
3       786.661056  310.018206    0.999839              0
4       786.950856  308.927856    0.999821              0


C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.7478858350951374, 0.67047223663726, 0.546795523906409, 0.6442917547568711, 0.0, 0.2540343571056741, 0.521783700666325, 0.7551799345692476, 0.8755102040816326, 0.5752577319587628]


In [48]:
x = zip(dfs, times, mouse_number, proportion_spent, frame_rate)
print(len(frames))
print(len(frame_rate))
df = pd.DataFrame(np.array([mouse_number, proportion_spent, times, frames, frame_rate]).T, columns=["mouse", "proportion_spent", "vid_length", 'num_frames', 'frame_rate(ffmpeg)'])
print('hi')
df['frame_rate(vid_length/num_frames)'] = df['num_frames'].values / df['vid_length'].values
df['time_spent'] = df['vid_length'].values * df['proportion_spent'].values
df.to_csv(r"C:\Users\vjj14\Desktop\DeepLabCut\watermaze-vj-2019-07-02\watermaze.csv")

10
10
hi


In [47]:
df

,mouse,proportion_spent,vid_length,num_frames,frame_rate(ffmpeg),frame_rate(vid_length/num_frames),time_spent
0,50909.0,0.747886,65.40,1892.0,28.92,28.929664,48.911734
1,50911.0,0.670472,65.45,1927.0,29.97,29.442322,43.882408
2,50944.0,0.546796,65.58,1966.0,30.00,29.978652,35.858850
3,56199.0,0.644292,65.46,1892.0,28.90,28.903147,42.175338
4,56242.0,0.000000,64.26,1926.0,30.00,29.971989,0.000000
5,56244.0,0.254034,65.44,1921.0,29.36,29.355134,16.624008
6,63410.0,0.521784,65.09,1951.0,30.00,29.973882,33.962901
7,63447.0,0.755180,67.49,1834.0,27.18,27.174396,50.967094
8,66019.0,0.875510,65.39,1960.0,30.00,29.974002,57.249612
9,66020.0,0.575258,64.73,1940.0,30.00,29.970647,37.236433


In [24]:
df['num_frames']/df['vid_length']

0    28.929664
1    29.442322
2    29.978652
3    28.903147
4    29.971989
5    29.355134
6    29.973882
7    27.174396
8    29.974002
9    29.970647
dtype: float64